In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from peft import PeftModel, get_peft_model_state_dict
from transformers import AutoModelForCausalLM
import torch
from huggingface_hub import login, HfApi, create_repo
from kaggle_secrets import UserSecretsClient

In [ ]:
base_model_name = 'vohuutridung/qwen3-1.7b-legal-pretrain'
mcq_adapter = 'vohuutridung/stage1-mcq'
nli_adapter = 'vohuutridung/stage1-nli-v1-3e'
sqa_adapter = 'vohuutridung/stage1-sqa-v1-3e'

repo_id = "vohuutridung/merged3-v9" # repo to push the merged adapter

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, mcq_adapter, adapter_name='mcq')
_ = model.load_adapter(nli_adapter, adapter_name='nli')
_ = model.load_adapter(sqa_adapter, adapter_name='sqa')

In [ ]:
adapters = ['mcq', 'nli','sqa']
weights = [1.0, 1.0, 1.5]
new_adapter_name = 'merge'
combination_type = 'dare_ties_svd'
majority_sign_method = 'frequency'
density = 0.8

model.add_weighted_adapter(
    adapters,
    weights,
    new_adapter_name,
    combination_type=combination_type,
    majority_sign_method=majority_sign_method,
    density=density,
)

In [ ]:
model.set_adapter(new_adapter_name)
model.delete_adapter('mcq')
model.delete_adapter('nli')
model.delete_adapter('sqa')

In [ ]:
for name, param in model.named_parameters():
    if "lora_" in name:
        param.data = param.data.contiguous()

In [ ]:
model.save_pretrained('merge_adapter')

In [ ]:
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
login(HF_TOKEN)
api = HfApi()

local_dir = f"merge_adapter/{new_adapter_name}"
create_repo(repo_id, exist_ok=True)

for filename in os.listdir(local_dir):
    local_path = os.path.join(local_dir, filename)

    if os.path.isfile(local_path):
        print(f"Uploading {filename} ...")
        api.upload_file(
            path_or_fileobj=local_path,
            path_in_repo=filename,
            repo_id=repo_id,
        )

print("Done!")

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

adapter_path = repo_id
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    is_trainable=True,
)
model.print_trainable_parameters()

In [ ]:
sd = get_peft_model_state_dict(model)
print(len(sd))